In [1]:
from google.colab import drive
drive.mount('/content/drive')#, force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install sentence_transformers
!pip install evaluate
!pip install bert_score
!pip install gradio==3.48.0
!pip install kaleido

In [3]:
from sentence_transformers import SentenceTransformer
model_emb = SentenceTransformer('BAAI/bge-large-en-v1.5')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [4]:
import pandas as pd
data = pd.read_csv("drive/MyDrive/Сертификат/news-dataset/assets/test.csv",header=None)
data = data.values
for i in range(len(data)):
    data[i][0]=i

In [5]:
all_embs =  []
from tqdm import tqdm
for el in tqdm(data):
    embs = model_emb.encode([el[2]], normalize_embeddings=True)
    all_embs.append((el[0],embs))

100%|██████████| 7600/7600 [03:39<00:00, 34.61it/s]


In [ ]:
!pip install chromadb

In [7]:
!python --version

Python 3.10.12


In [ ]:
!pip install --upgrade --force-reinstall fastapi==0.103.2

In [9]:
import chromadb

In [10]:

client = chromadb.Client()

collection = client.create_collection("cos_db", {"hnsw:space": "cosine"})
abc = [el[1][0].tolist() for el in all_embs]
collection.add(
    embeddings=abc,
    metadatas=[{"headline":el} for el in data[:,1].tolist()],
    ids=[str(el[0]) for el in all_embs]
)

In [11]:
def get_documents(q):
    instruction = "Represent this sentence for searching relevant passages:"
    query = instruction + " " + q
    query = model_emb.encode(query, normalize_embeddings=True)
    results = collection.query(query_embeddings=[query.tolist()],n_results=10)
    return results

In [12]:
def get_answer(question):
    docs = [data[int(el)][2] for el in get_documents(question)["ids"][0]]
    return docs

In [13]:
questions = ["Who are the founders of Microsoft?",
             "Who is russia president?",
            "Who is the mobile phone China leader?",
            "Who first developed the theory of evolution",
            'What political party is currently in power in the United States?',
             "Where President Bush meets with Italian Prime Minister Silvio Berlusconi?",
             "What is a record of global sales of mobile telephones?",
             "Who was George Bush?",
             'What is the currency of Japan?',
             'Who wrote the play "Romeo and Juliet"?']

answers = ["Bill Gates and Paul Allen",
           "Vladimir Putin",
          "Samsung",
          "Charles Darwin",
           'The Democratic Party',
           "White House",
           "156 mn.",
           'USA President',
           'yen',
           'William Shakespeare']

In [14]:
context1 = [get_answer(i) for i in questions]

In [15]:
get_answer("Who was George Bush?")

['Reuters - President Bush on Wednesday urged\\Russian President Vladimir Putin to "uphold the principles of\\democracy" in a carefully worded message expressing concern\\about Putin\'s proposed political reforms.',
 'AFP - Facing a issue that once tripped up his father, US President George W. Bush told adoring supporters that he likes broccoli. Part of it, anyway.',
 "Reuters - As George W. Bush formulates his plans\\for a Middle East without Yasser Arafat, British Prime Minister\\Tony Blair will be standing where he says he belongs: right by\\the U.S. president's side.",
 'BEIRUT: Consistently second only to Ariel Sharon in terms of unpopularity among Arabs, US President George W. Bush #39;s re-election victory was greeted in the Arab world with a sense of disillusionment and foreboding.',
 'AFP - President George W. Bush has rewritten US foreign policy during four years at the White House, with the war on terror now taking priority and doubt cast on some traditional alliances.',
 'B

In [16]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

from evaluate import load
bertscore = load("bertscore")


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [17]:
bs_all = []
for q, a, index in zip(questions, answers, range(len(answers))):
  QA_input = {'question': q,
             'context': ' '.join(context1[index])}
  res = nlp(QA_input)
  bs = bertscore.compute(predictions=[res['answer']], references=[a], lang="en")
  bs_all.append(bs)

  print(f'Question: {q}\nAnswer: {res["answer"]}\nTrue answer: {a}\nScore: {bs["f1"][0]}\n ')

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: Who are the founders of Microsoft?
Answer: Bill Gates
True answer: Bill Gates and Paul Allen
Score: 0.9312379956245422
 
Question: Who is russia president?
Answer: Mr Vladimir V. Putin
True answer: Vladimir Putin
Score: 0.9284749031066895
 
Question: Who is the mobile phone China leader?
Answer: Samsung Electronics
True answer: Samsung
Score: 0.868729293346405
 
Question: Who first developed the theory of evolution
Answer: Darwin
True answer: Charles Darwin
Score: 0.8472084403038025
 
Question: What political party is currently in power in the United States?
Answer: Democratic
True answer: The Democratic Party
Score: 0.8500853180885315
 
Question: Where President Bush meets with Italian Prime Minister Silvio Berlusconi?
Answer: Oval Office of the White House
True answer: White House
Score: 0.8881983757019043
 
Question: What is a record of global sales of mobile telephones?
Answer: 156 million
True answer: 156 mn.
Score: 0.935771107673645
 
Question: Who was George Bush?
Answ

In [18]:
f1_scores = [bs['f1'][0] for bs in bs_all]
sum(f1_scores)/len(f1_scores)

0.8838155269622803

In [19]:
import gradio as gr

def echo(question, history):
    QA_input = {'question': question,
             'context': ' '.join(context1[1])}
    res = nlp(QA_input)
    return res['answer']

demo = gr.ChatInterface(fn=echo, examples=["hello", "hola", "merhaba"], title="Echo Bot")
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6105613281b93f0f49.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
